In [1]:
import pandas as pd
import numpy as np


from sqlalchemy import create_engine


ck_properties = {  # clickhouse 数据库配置
    # "driver": "com.github.housepower.jdbc.ClickHouseDriver",
    "driver": "ru.yandex.clickhouse.ClickHouseDriver",
    "socket_timeout": "300000",
    "rewriteBatchedStatements": "true",
    "batchsize": "1000000",
    "numPartitions": "8",
    "user": "default",
    "password": "5NsWWQeJ",
    "host": "10.129.165.72",
    "database": 'sdecdmp',
    "port": 8123
}


mysql_properties_prod = {  # mysql 生产环境配置
    "host": "10.129.41.147",
    "port": "4918",
    "url": "jdbc:mysql://10.129.41.147:4918/ck",
    "user": "dmp_sys_guest",
    "password": "fTe^3hy*dO_363",
    "driver": "com.mysql.cj.jdbc.Driver",
    "database": "ck"
}

mysql_properties_dev = {  # mysql 开发环境配置
    "host": "10.179.98.13",
    "port": "3306",
    "url": "jdbc:mysql://10.179.98.13:3306/ck",
    "user": "root",
    "password": "DMP_test135#",
    "driver": "com.mysql.cj.jdbc.Driver",
    "database": "ck"
}

ck_engine = create_engine(
    f"clickhouse://{ck_properties['user']}:{ck_properties['password']}@{ck_properties['host']}:{ck_properties['port']}/{ck_properties['database']}"
)

mysql_engine_prod = create_engine(
    f"mysql+pymysql://{mysql_properties_prod['user']}:{mysql_properties_prod['password']}" +
    f"@{mysql_properties_prod['host']}:{mysql_properties_prod['port']}/{mysql_properties_prod['database']}?charset=utf8"
)

mysql_engine_dev = create_engine(
    f"mysql+pymysql://{mysql_properties_dev['user']}:{mysql_properties_dev['password']}" +
    f"@{mysql_properties_dev['host']}:{mysql_properties_dev['port']}/{mysql_properties_dev['database']}?charset=utf8"
)


In [2]:
import os
os.path.abspath('.')

'/data1/pythonProjects/pyetl/core/notebooks'

In [3]:
# 读取NOTCCPPARAINFO
para_df = pd.read_sql(
    sql=f"select * from ods.eol_notccpparainfo",
    con=mysql_engine_prod
)
para_df.head()

,ID,PID,ECUName,AcqstEnble,GetMode,UploadEnable,Uploadmode,Uploadtime,Channel,Style,...,dtcmaxnum,curdtcmark,Bz1,Bz2,pkgname,maxval,minval,active,group_code,signal_type
0,25077,C81_U010100,C81,1,2,1,1,1,1,1,...,0,0,,Active,DID0101,,,Y,7001,signed_int16
1,25078,C81_U010300,C81,1,2,1,1,1,1,1,...,0,0,,Active,DID0103,,,Y,7001,signed_int16
2,25079,C81_U010400,C81,1,2,1,1,1,1,1,...,0,0,,Active,DID0104,,,Y,7001,unsigned_int
3,25080,C81_U010500,C81,1,2,1,1,1,1,1,...,0,0,,Active,DID0105,,,Y,7001,signed_int16
4,25081,C81_U010600,C81,1,2,1,1,1,1,1,...,0,0,,Active,DID0106,,,Y,7001,signed_int16


In [4]:
# 读取已经整理的功能 PID 对应关系
app_pid_df = pd.read_excel(
    "/home/qjy20472/pythonProjects/pyetl/data/研发项目管理.xlsx",
    sheet_name="模型信号清单表"
)
app_pid_df = app_pid_df.loc[:, ['统一信号名(无单位)', 'Confirm PIDList']]
app_pid_df.columns = ['target_name', 'pids']
app_pid_df

/home/qjy20472/miniconda3/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,target_name,pids
0,EIN,/
1,VIN,/
2,标定数据零件号,/
3,标定数据版本号,/
4,软件标定识别号,/
...,...,...
140,尿素泵温度,NaN
141,GOC入口温度,SAE_J004753
142,GOC出口温度,SAE_J004754
143,总气耗,SAE_J520504


In [5]:
# 读取etl_map_all
etl_map_df = pd.read_sql(
    sql=f"SELECT * FROM dmp.etl_map_all",
    con=mysql_engine_prod
)
etl_map_df

,id,target_desc,target_name,signal_code,target_unit,signal_unit,signal_type,data_source,platname,hos_series,data_type,hos_mat_id,extend_feature,multiplier,offset,create_date,create_time,update_time
0,1,gps经度,lng,longitude,°,°,None,snat,大通,E,gc,None,None,None,None,None,2023-07-21 11:03:09,2023-07-21 11:03:09
1,2,gps纬度,lat,latitude,°,°,None,snat,大通,E,gc,None,None,None,None,None,2023-07-21 11:03:09,2023-07-21 11:03:09
2,3,gps海拔高度,high,None,km,None,None,snat,大通,E,gc,None,None,None,None,None,2023-07-21 11:03:09,2023-07-21 11:03:09
3,4,大气压力,atmospheric_pressure,SAE_J000108,kpa,kpa,None,snat,大通,E,gc,None,None,None,None,None,2023-07-21 11:03:09,2023-07-21 11:03:09
4,5,大气温度,atmospheric_temperature,SAE_J000171,℃,℃,None,snat,大通,E,gc,None,None,None,None,None,2023-07-21 11:03:09,2023-07-21 11:03:09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313,314,t6(SCR入口温度),t6_temperature,SAE_J004360;SAE_J003241;SAE_A003241,℃,℃;℃;℃,None,snat,大通,all,gc,None,None,None,None,None,2023-08-01 18:51:15,2023-08-01 18:51:15
314,315,节气门开度,throttle_position,SAE_J000051,%,%,None,snat,大通,all,gc,None,None,None,None,None,2023-08-01 18:51:15,2023-08-01 18:51:15
315,316,累计里程,total_distance,SAE_J000245,km,km,None,snat,大通,all,gc,None,None,None,None,None,2023-08-01 18:51:15,2023-08-01 18:51:15
316,317,累计油耗,total_fuel,SAE_J000250,L,L,None,snat,大通,all,gc,None,None,None,None,None,2023-08-01 18:51:15,2023-08-01 18:51:15


In [6]:
# 对已经整理的信号清单，分割pid，匹配中文名
app_pid_flatten_list = []
for _, row in app_pid_df.iterrows():
    if row['pids'] == "/":
        # 无对应pid跳过该条
        continue

    if isinstance(row['pids'], float):
        continue

    for pid in row['pids'].split('、'):
        app_pid_flatten_list.append([row['target_name'], pid])
app_pid_flatten_df = pd.DataFrame(app_pid_flatten_list, columns=['target_desc', 'pid'])
app_pid_flatten_df

,target_desc,pid
0,PTO油门,DCW_U010314
1,离合器开关,SAE_J000598
2,空档开关,DCW_U010202
3,空档开关,MD1_U014500
4,空调开关,DCW_U010203
...,...,...
186,SCR子状态,V51_U160403
187,SCR子状态,MD1_U124C00
188,GOC入口温度,SAE_J004753
189,GOC出口温度,SAE_J004754


In [7]:
# 对etl_map_all数据，分割pid
etl_map_flatten_list = []
for _, row in etl_map_df.iterrows():
    if row['signal_code'] is None:
        continue

    for pid in row['signal_code'].split(';'):
        etl_map_flatten_list.append([row['target_desc'], row['target_name'], row['hos_series'], pid])
etl_map_flatten_df = pd.DataFrame(etl_map_flatten_list, columns=['target_desc', 'target_name', 'hos_series', 'pid'])
etl_map_flatten_df

,target_desc,target_name,hos_series,pid
0,gps经度,lng,E,longitude
1,gps纬度,lat,E,latitude
2,大气压力,atmospheric_pressure,E,SAE_J000108
3,大气温度,atmospheric_temperature,E,SAE_J000171
4,车速,speed,E,SAE_J000084
...,...,...,...,...
358,t6(SCR入口温度),t6_temperature,all,SAE_A003241
359,节气门开度,throttle_position,all,SAE_J000051
360,累计里程,total_distance,all,SAE_J000245
361,累计油耗,total_fuel,all,SAE_J000250


In [8]:
merged_df = pd.merge(
    left=app_pid_flatten_df,
    right=etl_map_flatten_df,
    on='pid',
    how='left'
)
merged_df

,target_desc_x,pid,target_desc_y,target_name,hos_series
0,PTO油门,DCW_U010314,NaN,NaN,NaN
1,离合器开关,SAE_J000598,NaN,NaN,NaN
2,空档开关,DCW_U010202,NaN,NaN,NaN
3,空档开关,MD1_U014500,NaN,NaN,NaN
4,空调开关,DCW_U010203,NaN,NaN,NaN
...,...,...,...,...,...
458,SCR子状态,V51_U160403,NaN,NaN,NaN
459,SCR子状态,MD1_U124C00,NaN,NaN,NaN
460,GOC入口温度,SAE_J004753,NaN,NaN,NaN
461,GOC出口温度,SAE_J004754,NaN,NaN,NaN


In [9]:
# 找出V51开头的PID信息
merged_df.loc[merged_df['pid'].str.contains('V51')]

,target_desc_x,pid,target_desc_y,target_name,hos_series
451,DCU故障码,V51_U190201,NaN,NaN,NaN
452,尿素喷嘴占空比,V51_U162001,NaN,NaN,NaN
454,尿素泵占空比,V51_U162002,NaN,NaN,NaN
456,SCR主状态,V51_U160401,NaN,NaN,NaN
458,SCR子状态,V51_U160403,NaN,NaN,NaN


# 单位统一辅助代码

In [10]:
pid_desc_df = pd.read_excel(
    '/home/qjy20472/pythonProjects/pyetl/data/PID统计.xlsx', sheet_name='PID', header=1
)
pid_desc_df

/home/qjy20472/miniconda3/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,pid,Description,target_desc,修改标记,target_name,youdao_result,target_unit,unit,app,待确认单位
0,SAE_J520534,闭环系数,闭环系数,NaN,closed_loop_coefficient,closed_loop_coefficient,-,-,NaN,NaN
1,C81_U022C00,部分再生成功次数,部分再生成功次数,NaN,part_regeneration_success_counter,number_of_successful_partial_regeneration,-,-,NaN,NaN
2,MD1_U114B00,部分再生成功次数,部分再生成功次数,NaN,part_regeneration_success_counter,number_of_successful_partial_regeneration,-,-,NaN,NaN
3,MD1_U013000,挡位,挡位,NaN,gear,shift,-,-,路谱报表,NaN
4,SAE_J000523,挡位,挡位,NaN,gear,shift,-,-,路谱报表,NaN
...,...,...,...,...,...,...,...,...,...,...
339,DCW_U010A04,G激活,G激活,NaN,g_activate,g_activate,NaN,-,NaN,NaN
340,C81_U018300,OBD灯状态,OBD灯状态,NaN,obd_indicator_status,obd_indicator_status,NaN,-,NaN,NaN
341,WW_U01241E,PTO开关,PTO开关,NaN,pto,pto_switch,NaN,-,NaN,NaN
342,V51_U160401,SCR主状态,SCR主状态,NaN,scr_main_status,scr_main_status,NaN,NaN,NaN,NaN


In [11]:
# 压强
pattern = r'\b(hpa|mpa|kpa)\b'
pressure_pid_df = pid_desc_df.loc[pid_desc_df['target_unit'].str.contains(pattern, case=False, na=False)]
pressure_pid_df = pressure_pid_df.loc[:, ['Description', 'target_desc', 'target_unit']]
pressure_pid_df.drop_duplicates(inplace=True)
pressure_pid_df

<ipython-input-11-14b56cc6e728>:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  pressure_pid_df = pid_desc_df.loc[pid_desc_df['target_unit'].str.contains(pattern, case=False, na=False)]


,Description,target_desc,target_unit
177,环境压力,大气压力,hpa
178,轨压偏差,轨压偏差,hpa
179,尿素泵压力,尿素泵压力,hpa
181,DPF流阻,DPF流阻,hpa
182,发动机进气压力,发动机进气压力,hPa
200,大气压力,大气压力,kpa
201,喉口压力,喉口压力,kpa
202,机油压力,机油压力,kpa
203,进气歧管压力,进气歧管压力,kpa
205,目标增压压力,目标增压压力,kpa


In [12]:
# 空气流量
pattern = r'(m3/h|L/h|kg/h)'
air_flow_pid_df = pid_desc_df.loc[pid_desc_df['target_unit'].str.contains(pattern, case=False, na=False)]
air_flow_pid_df = air_flow_pid_df.loc[:, ['Description', 'target_desc', 'target_unit']]
air_flow_pid_df.drop_duplicates(inplace=True)
air_flow_pid_df

<ipython-input-12-e7f942a991a0>:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  air_flow_pid_df = pid_desc_df.loc[pid_desc_df['target_unit'].str.contains(pattern, case=False, na=False)]


,Description,target_desc,target_unit
185,发动机进气量,发动机进气量,kg/h
186,发动机燃气流量,发动机燃气流量,kg/h
187,废气质量流量,废气质量流量,kg/h
188,进气流量,进气流量,kg/h
189,进气流量漂移补偿,进气流量漂移补偿,kg/h
190,目标进气量,目标进气量,kg/h
224,发动机燃料流量,发动机燃料流量,L/h
230,排气流量,排气流量,m3/h
232,循环进气量,循环进气量,m3/h
233,循环EGR进气量,循环进气量,m3/h


In [13]:
# 液体喷射量
pattern = r'(mm3/st|g/min|mg/hub|mg/s)'
liquid_injection_pid_df = pid_desc_df.loc[pid_desc_df['unit'].str.contains(pattern, case=False, na=False)]
liquid_injection_pid_df = liquid_injection_pid_df.loc[:, ['Description', 'target_desc', 'target_unit']]
liquid_injection_pid_df.drop_duplicates(inplace=True)
liquid_injection_pid_df

<ipython-input-13-818227a05f3d>:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  liquid_injection_pid_df = pid_desc_df.loc[pid_desc_df['unit'].str.contains(pattern, case=False, na=False)]


,Description,target_desc,target_unit
173,尿素喷射量指令,尿素喷射量,g/min
174,尿素喷射量,尿素喷射量,g/min
175,实际HCI喷油量,实际HCI喷油量,g/min
236,每气缸进气量,每气缸进气量,mg/hub
237,喷油量设定值,喷油量设定值,mg/hub
238,喷油量自学习修正值,喷油量自学习修正值,mg/hub
239,当前喷油量,实时喷油量,mg/hub
240,喷油量,实时喷油量,mg/hub
241,实时喷油量,实时喷油量,mg/hub
243,总Post喷油量,总Post喷油量,mg/hub
